In [3]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [6]:
client.create_experiment(name = 'yet-another-experiment')

'2'

In [7]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = '1',
    filter_string = 'metrics.rmse < 7',
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ["metrics.rmse ASC"]
)

In [14]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
run_ids = []
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")
    run_ids.append(run.info.run_id)

run id: 250fe19f833b48f9badcd00b4e532ef9, rmse: 6.1413
run id: cd65b4652cd94e68b83bda883b418e28, rmse: 6.1413
run id: 7c8b265514f549a2a9cc87a319ef91a1, rmse: 6.1413
run id: a2b0c81a74184256b1cd09b02aa99438, rmse: 6.1463
run id: e38663faeab249f884e39682f88f2cde, rmse: 6.2267


In [16]:
for run_id in run_ids:
    model_uri = f'runs:/{run_id}/model'
    mlflow.register_model(model_uri = model_uri, name = 'nyc-taxi-regressor')

Successfully registered model 'nyc-taxi-regressor'.
2022/05/30 17:19:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 1
Created version '1' of model 'nyc-taxi-regressor'.
Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/30 17:19:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.
Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/30 17:19:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.
Registered model 'nyc-taxi-regress

In [25]:
model_name = 'nyc-taxi-regressor'
version_stages = client.get_latest_versions(name = model_name)

for version in version_stages:
    print(f'version : {version.version}, stage : {version.current_stage}')

version : 4, stage : None
version : 5, stage : Staging


#### **Change model stage**

In [45]:
model_version = 1
new_stage = 'Production' # Staged first
client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions = False
)

<ModelVersion: creation_timestamp=1653931192603, current_stage='Production', description='The model version 1 was transitioned to Staging on 2022-05-30', last_updated_timestamp=1653935486406, name='nyc-taxi-regressor', run_id='250fe19f833b48f9badcd00b4e532ef9', run_link=None, source='./mlruns/1/250fe19f833b48f9badcd00b4e532ef9/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

#### **Update version**

In [46]:
from datetime import datetime


date = datetime.today().date()
client.update_model_version(
    name = model_name,
    version = model_version,
    description = f'The model version {model_version} was transitioned to {new_stage} on {date}'
)

<ModelVersion: creation_timestamp=1653931192603, current_stage='Production', description='The model version 1 was transitioned to Production on 2022-05-30', last_updated_timestamp=1653935503020, name='nyc-taxi-regressor', run_id='250fe19f833b48f9badcd00b4e532ef9', run_link=None, source='./mlruns/1/250fe19f833b48f9badcd00b4e532ef9/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

#### **Comparing versions and selecting the new "Production" model**

In [28]:
import pandas as pd
from sklearn.metrics import mean_squared_error

In [29]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df[categorical] = df[categorical].astype(str)
    
    return df

In [41]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [42]:
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [49]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [56]:
run_id='250fe19f833b48f9badcd00b4e532ef9' #xgboost model id, not the latest model

In [57]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/zoomcampdataengineering/mlops_zoomcamp/2-Experiment_Tracking/preprocessor'

In [58]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [59]:
X_test = preprocess(df, dv)

In [60]:
target = "duration"
y_test = df[target].values

In [63]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[18:38:26] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 5.61 s, sys: 48.4 ms, total: 5.66 s
Wall time: 2.08 s


{'rmse': 6.711533398532478}